In [1]:
import re
import time
import datetime
import requests
import pandas as pd
import multiprocessing
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from Functions.funciones import movies_for_category, categories, print_process, movies, series_informacion, desempaquetado, generador_dummies

# -------------- Parametros --------------
ruta_base = "Bases_datos/"

# ----------------------------------------

In [2]:
print_process("Comienzo del Notebook 'Scraping'")

|-----------------------------Comienzo del Script-----------------------------|


In [3]:
categorias = categories()

|---------------------Categorias capturadas correctamente.---------------------|
|-----------------------Categorias demoro 11.77 segundos-----------------------|


In [4]:

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {}

categorias_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies_for_category, categorias))
    
    # Asignar los resultados al diccionario
    for categoria, resultado in zip(categorias, resultados):
        resultados_dict[categoria] = resultado

peliculas_total = []

for x,i in resultados_dict.items():
    for e in i:
        peliculas_total.append(e)

peliculas_total = set(peliculas_total)

categorias_fin = time.time()

print_process(f"La extraccion total de las categorias tubo una demora de {round((categorias_fin-categorias_comienzo),2)} segundos")


|Todo el contenido de la categoria 'history' fue capturado correctamente, demoro 27.8 segundos|
|Todo el contenido de la categoria 'biography' fue capturado correctamente, demoro 28.38 segundos|
|Todo el contenido de la categoria 'fantasy' fue capturado correctamente, demoro 28.5 segundos|
|Todo el contenido de la categoria 'crime' fue capturado correctamente, demoro 28.87 segundos|
|Todo el contenido de la categoria 'family' fue capturado correctamente, demoro 29.12 segundos|
|Todo el contenido de la categoria 'action' fue capturado correctamente, demoro 29.9 segundos|
|Todo el contenido de la categoria 'animation' fue capturado correctamente, demoro 31.57 segundos|
|Todo el contenido de la categoria 'horror' fue capturado correctamente, demoro 40.23 segundos|
|Todo el contenido de la categoria 'news' fue capturado correctamente, demoro 13.3 segundos|
|Todo el contenido de la categoria 'adventure' fue capturado correctamente, demoro 48.99 segundos|
|Todo el contenido de la categoria '

In [5]:
peliculas_total = [x for x in peliculas_total if "/content/" in x]

In [6]:
print_process("Comienzo del proceso de extraccion de informacion de cada iteracion")

|-----Comienzo del proceso de extraccion de informacion de cada iteracion-----|


In [7]:
# Crear un diccionario para almacenar los resultados
todo = {"titulo": [],"idioma" : [], "subtitulos" : [], "ano": [], "duracion_minutos": [], "categorias": [], "sinopsis": [], "tipo": [], "link": []}

comienzo_movies = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies, peliculas_total))

    try:
        for resultado in resultados:
            for clave, valor in resultado.items():
                todo[clave].append(valor)
    except:
        print()

fin_movies = time.time()

print_process(f"El proceso de captura de la informacion de todo el contenido fue exsitoso, y demoro {round((fin_movies - comienzo_movies) / 60,2)} minutos")


Error al cargar la url https://filmzie.com/content/12-days-with-god-2019 en el intento 1: 404
|El proceso de captura de la informacion de todo el contenido fue exsitoso, y demoro 343.48 segundos|


In [8]:
df = pd.DataFrame(todo)

In [14]:

df.to_csv(f"{ruta_base}Peliculas_Series_Completo_Respaldo.csv" , index=False)

In [2]:
df = pd.read_csv(f"{ruta_base}Peliculas_Series_Completo_Respaldo.csv")

In [3]:
peliculas = df.loc[df["tipo"] == "Pelicula"].reset_index(drop=True)
series = df.loc[df["tipo"] == "Serie"].reset_index(drop=True)

In [4]:

peliculas["duracion_minutos"] = peliculas["duracion_minutos"].str.replace(" min","").astype(int)
peliculas.loc[peliculas["idioma"] == 'Non linguistic content', "idioma"] = "No contiene idioma"
peliculas["subtitulos"] = peliculas["subtitulos"].fillna("No contiene subtitulos")


In [5]:
peliculas

,titulo,idioma,subtitulos,ano,duracion_minutos,categorias,sinopsis,tipo,link
0,Casimira,Spanish,English (embedded),2020,23,"Short, Drama","In a small, isolated farmhouse in Patagonia, C...",Pelicula,https://filmzie.com/content/casimira-2020
1,Mission Star,Spanish,"English, Portuguese",2017,104,"Action, Comedy, Drama",A delivery boy from a grocery store gets the d...,Pelicula,https://filmzie.com/content/mission-star-2017
2,Real Cases of Shadow People: The Sarah McCormi...,English,English,2019,105,"Horror, Mystery, Thriller",Mysterious disappearances. Missing students ca...,Pelicula,https://filmzie.com/content/real-cases-of-shad...
3,Big Noise Dispatches 07,English,English,2011,34,"Documentary, Short","Crisis in counterinsurgency, US forces in Iraq...",Pelicula,https://filmzie.com/content/big-noise-dispatch...
4,Fighting Fish,English,English,2010,83,Drama,David's sister's return after years triggers a...,Pelicula,https://filmzie.com/content/fighting-fish-2010
...,...,...,...,...,...,...,...,...,...
2199,Momma Said Come Home for Christmas,English,English,2023,81,Comedy,The matriarch of a family calls all her childr...,Pelicula,https://filmzie.com/content/momma-said-come-ho...
2200,Train Set,English,No contiene subtitutlos,2018,96,"Comedy, Drama, Romance",Sam's happy family life takes a dark turn when...,Pelicula,https://filmzie.com/content/train-set
2201,Iraq: The Wind of Hope,English,English,2013,90,Documentary,Experience the deeply personal journey of Iraq...,Pelicula,https://filmzie.com/content/iraq-the-wind-of-h...
2202,The Way Home,English,English,2010,90,"Drama, Family",An inspirational true story about how a rural ...,Pelicula,https://filmzie.com/content/the-way-home-2010


In [6]:
peliculas.to_csv(f"{ruta_base}Peliculas.csv",index=False)

In [7]:
peliculas_dummies = generador_dummies(peliculas)

In [8]:
# Exportamos base de datos de Peliculas con dummies
peliculas_dummies.to_csv(f"{ruta_base}Peliculas_dummies.csv",index=False)

,titulo,idioma,subtitulos,ano,duracion_minutos,sinopsis,tipo,link,Fantasy,Mystery,...,Documentary,Sport,Adventure,War,Musical,Drama,Comedy,Reality TV,Short,Animation
0,Casimira,Spanish,English (embedded),2020,23,"In a small, isolated farmhouse in Patagonia, C...",Pelicula,https://filmzie.com/content/casimira-2020,0,0,...,0,0,0,0,0,1,0,0,1,0
1,Mission Star,Spanish,"English, Portuguese",2017,104,A delivery boy from a grocery store gets the d...,Pelicula,https://filmzie.com/content/mission-star-2017,0,0,...,0,0,0,0,0,1,1,0,0,0
2,Real Cases of Shadow People: The Sarah McCormi...,English,English,2019,105,Mysterious disappearances. Missing students ca...,Pelicula,https://filmzie.com/content/real-cases-of-shad...,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Big Noise Dispatches 07,English,English,2011,34,"Crisis in counterinsurgency, US forces in Iraq...",Pelicula,https://filmzie.com/content/big-noise-dispatch...,0,0,...,1,0,0,0,0,0,0,0,1,0
4,Fighting Fish,English,English,2010,83,David's sister's return after years triggers a...,Pelicula,https://filmzie.com/content/fighting-fish-2010,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2199,Momma Said Come Home for Christmas,English,English,2023,81,The matriarch of a family calls all her childr...,Pelicula,https://filmzie.com/content/momma-said-come-ho...,0,0,...,0,0,0,0,0,0,1,0,0,0
2200,Train Set,English,No contiene subtitutlos,2018,96,Sam's happy family life takes a dark turn when...,Pelicula,https://filmzie.com/content/train-set,0,0,...,0,0,0,0,0,1,1,0,0,0
2201,Iraq: The Wind of Hope,English,English,2013,90,Experience the deeply personal journey of Iraq...,Pelicula,https://filmzie.com/content/iraq-the-wind-of-h...,0,0,...,1,0,0,0,0,0,0,0,0,0
2202,The Way Home,English,English,2010,90,An inspirational true story about how a rural ...,Pelicula,https://filmzie.com/content/the-way-home-2010,0,0,...,0,0,0,0,0,1,0,0,0,0


In [9]:
series.drop(columns=['duracion_minutos'], inplace=True)
series

,titulo,idioma,subtitulos,ano,categorias,sinopsis,tipo,link
0,UFO Events: The Best of Third Phase,English,Subtitles,2020,Documentary,Watch our Special reports on UFO related histo...,Serie,https://filmzie.com/content/ufo-events-the-bes...
1,The Psychic,English,Subtitles,2019,Animation,The cave was the big challenge. An almost insu...,Serie,https://filmzie.com/content/the-psychic-2019
2,C-Bar,English,Subtitles,2015,Western,After a young girl's family is brutally murder...,Serie,https://filmzie.com/content/c-bar-2015
3,The Past Hunters,English,Subtitles,2015,"Horror, European Production",The team delve deep into some of the UK's most...,Serie,https://filmzie.com/content/the-past-hunters-2015
4,The Paraseekers,English,Subtitles,2018,Reality TV,Join the team investigating Corrados farm's ha...,Serie,https://filmzie.com/content/the-paraseekers-2018
...,...,...,...,...,...,...,...,...
94,The Witching Season,English,Subtitles,2015,Horror,An anthology of chilling short horror films se...,Serie,https://filmzie.com/content/the-witching-seaso...
95,Heroes Manufactured: Creators Unleashed,English,Subtitles,2020,Reality TV,A reality series following North America's top...,Serie,https://filmzie.com/content/heroes-manufacture...
96,Threesome,English,Subtitles,2015,Comedy,Vanessa and Justin are your everyday couple ex...,Serie,https://filmzie.com/content/threesome-2015
97,Ladies That Punch!,English,Subtitles,2020,"Documentary, Sport","Watch as catwalk models, teachers, and nurses ...",Serie,https://filmzie.com/content/ladies-that-punch-...


In [8]:
links_series = list(series["link"].values)
links_series

['https://filmzie.com/content/ufo-events-the-best-of-third-phase-2020',
 'https://filmzie.com/content/the-psychic-2019',
 'https://filmzie.com/content/c-bar-2015',
 'https://filmzie.com/content/the-past-hunters-2015',
 'https://filmzie.com/content/the-paraseekers-2018',
 'https://filmzie.com/content/we-are-all-human-2020',
 'https://filmzie.com/content/animal-armory-2016',
 'https://filmzie.com/content/i-love-christmas-parties-2020',
 'https://filmzie.com/content/combat-report-2015',
 'https://filmzie.com/content/assassins-2020',
 'https://filmzie.com/content/missing-kenley-2021',
 'https://filmzie.com/content/the-foodie-2020',
 'https://filmzie.com/content/the-beech-boys-2019',
 'https://filmzie.com/content/new-zealand-from-above-2012',
 'https://filmzie.com/content/the-ave-2015',
 'https://filmzie.com/content/the-shadow-dimension-2021',
 'https://filmzie.com/content/bulge-bracket-2020',
 'https://filmzie.com/content/the-inflight-food-trip-2019',
 'https://filmzie.com/content/wilderne

In [45]:
procesar = links_series

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {"titulo": [], "temporadas": [], "capitulos": [], "sinopsis" : [], "duracion_minutos" : []}
# diccionario = {"titulo" : alm_3[0], "temporada" : alm_3[1], "capitulos" : alm_3[2], "sinopsis" : alm_3[3], "duracion_minutos" : alm_3[4]}
series_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados_temporales = list(executor.map(series_informacion, procesar))

# Separar los resultados temporales en listas individuales
titulos, temporadas, capitulos, sinopsis, duracion_minutos = zip(*resultados_temporales)

# Agregar los resultados a los diccionarios
resultados_dict["titulo"].extend(titulos)
resultados_dict["temporadas"].extend(temporadas)
resultados_dict["capitulos"].extend(capitulos)
resultados_dict["sinopsis"].extend(sinopsis)
resultados_dict["duracion_minutos"].extend(duracion_minutos)

series_fin = time.time()

print_process(f"La extraccion total de informacion de cada serie tuvo una demora de {round((series_fin-series_comienzo)/ 60, 2) } minutos")

Tiempo total de The Psychic: 23.94 segundos
Tiempo total de UFO Events: The Best of Third Phase: 46.9 segundos
|La extraccion total de informacion de cada serie tuvo una demora de 0.85 minutos|


In [61]:
df_2 = pd.DataFrame(desempaquetado(resultados_dict))
df_2

,titulo,temporadas,capitulos,sinopsis,duracion_minutos
0,UFO Events: The Best of Third Phase,Season 1,1. Huge Diamond Shaped UFO,"In this analysis, we are going to analyze Rich...",17
1,UFO Events: The Best of Third Phase,Season 1,2. Michio Kaku and The Pentagon Release,"In this special report, we have astrophysicist...",49
2,UFO Events: The Best of Third Phase,Season 1,3. UFO or CGI?,"In this special report, we are going to analyz...",17
3,UFO Events: The Best of Third Phase,Season 1,4. I Assaulted an Alien,"In this special report, we are going to discus...",16
4,UFO Events: The Best of Third Phase,Season 1,5. Military Exercise,"In this special report, we are going to show y...",10
5,UFO Events: The Best of Third Phase,Season 1,6. Trump Demands Transparency,In this special report we are going to talk ab...,16
6,UFO Events: The Best of Third Phase,Season 1,7. Pentagon Release of Off World Craft,"In this analysis, we are going to discuss with...",93
7,UFO Events: The Best of Third Phase,Season 1,8. Police Encounter,"In this episode, we are going to be looking at...",16
8,UFO Events: The Best of Third Phase,Season 1,9. Alien Archive,"In this special report, we are talking about t...",14
9,UFO Events: The Best of Third Phase,Season 1,10. Crashed UFO in the UK,"In this special report, we are going to talk a...",17


In [66]:
series_descripcion = pd.merge(series,df_2, on="titulo", how="inner")

series_descripcion

,titulo,idioma,subtitulos,ano,categorias,sinopsis_x,tipo,link,temporadas,capitulos,sinopsis_y,duracion_minutos
0,UFO Events: The Best of Third Phase,English,Subtitles,2020,Documentary,Watch our Special reports on UFO related histo...,Serie,https://filmzie.com/content/ufo-events-the-bes...,Season 1,1. Huge Diamond Shaped UFO,"In this analysis, we are going to analyze Rich...",17
1,UFO Events: The Best of Third Phase,English,Subtitles,2020,Documentary,Watch our Special reports on UFO related histo...,Serie,https://filmzie.com/content/ufo-events-the-bes...,Season 1,2. Michio Kaku and The Pentagon Release,"In this special report, we have astrophysicist...",49
2,UFO Events: The Best of Third Phase,English,Subtitles,2020,Documentary,Watch our Special reports on UFO related histo...,Serie,https://filmzie.com/content/ufo-events-the-bes...,Season 1,3. UFO or CGI?,"In this special report, we are going to analyz...",17
3,UFO Events: The Best of Third Phase,English,Subtitles,2020,Documentary,Watch our Special reports on UFO related histo...,Serie,https://filmzie.com/content/ufo-events-the-bes...,Season 1,4. I Assaulted an Alien,"In this special report, we are going to discus...",16
4,UFO Events: The Best of Third Phase,English,Subtitles,2020,Documentary,Watch our Special reports on UFO related histo...,Serie,https://filmzie.com/content/ufo-events-the-bes...,Season 1,5. Military Exercise,"In this special report, we are going to show y...",10
5,UFO Events: The Best of Third Phase,English,Subtitles,2020,Documentary,Watch our Special reports on UFO related histo...,Serie,https://filmzie.com/content/ufo-events-the-bes...,Season 1,6. Trump Demands Transparency,In this special report we are going to talk ab...,16
6,UFO Events: The Best of Third Phase,English,Subtitles,2020,Documentary,Watch our Special reports on UFO related histo...,Serie,https://filmzie.com/content/ufo-events-the-bes...,Season 1,7. Pentagon Release of Off World Craft,"In this analysis, we are going to discuss with...",93
7,UFO Events: The Best of Third Phase,English,Subtitles,2020,Documentary,Watch our Special reports on UFO related histo...,Serie,https://filmzie.com/content/ufo-events-the-bes...,Season 1,8. Police Encounter,"In this episode, we are going to be looking at...",16
8,UFO Events: The Best of Third Phase,English,Subtitles,2020,Documentary,Watch our Special reports on UFO related histo...,Serie,https://filmzie.com/content/ufo-events-the-bes...,Season 1,9. Alien Archive,"In this special report, we are talking about t...",14
9,UFO Events: The Best of Third Phase,English,Subtitles,2020,Documentary,Watch our Special reports on UFO related histo...,Serie,https://filmzie.com/content/ufo-events-the-bes...,Season 1,10. Crashed UFO in the UK,"In this special report, we are going to talk a...",17
